In [1]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import config
from db_manager import DBManager
import os
from rest_api import Api
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from urllib.parse import quote_plus
from sqlalchemy import create_engine
import itertools
import logging
import pickle
import hashlib

c:\Users\afroj\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\afroj\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
logging.basicConfig(
    format='%(levelname)s-%(lineno)s-%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level='INFO'
)
logger=logging.getLogger(__name__)

#### Show all columns without truncating

Today's date

In [3]:
today = date.today()
logger.info(f'Current date: {today}')

INFO-__main__:2-Current date: 2023-11-04


Adding months to today's date. Near term travel dates 30 - 60 days ahead

In [4]:
departure_date_1_month = today + relativedelta(months=1)
departure_date_2_month = today + relativedelta(months=2)
return_date_1_month = departure_date_2_month
return_date_2_month = today + relativedelta(months=3)
logger.info(f'Departure_dates:{departure_date_1_month} and {departure_date_2_month} \nReturn dates:{return_date_1_month} and {return_date_2_month}')

INFO-__main__:5-Departure_dates:2023-12-04 and 2024-01-04 
Return dates:2024-01-04 and 2024-02-04


Get the nearby festival departure dates by start_date from the public.indian_holidays table

In [10]:
# credentials=config.pg_credentials
# db = DBManager(credentials=credentials)
# df_departure_arrival_dates = db.run_query(query_file_name='festival_dates')

In [11]:
#df_departure_arrival_dates 

Apply the combination of params to get data for all possible combinations of depart, arrival dates and depart arrival airports

Combination itertools usage. Im calculating the total combinations by multiplying the number of possibilities (unique values) for each variable<br />
Departure airport: 2 possibilities<br />
Arrival airport: 4 possibilities<br />
Departure date: 2 possibilities<br />
Arrival date: 1 possibilities<br />
Total combinations = 2 x 4 x 2 x 1 = 16 combinations

#### Travel payouts API to get flight search results for each combination in the list 

--------------------------------------------------------

API test for amadeus

POST request to the Amadeus authorization server to get the access token

In [5]:
response = requests.post(url=config.url_token, headers=config.headers_token, data=config.data)

if response.status_code == 200:
  # API call succeeded
  token = response.json()['access_token'] 
  logger.info('Successful access token generation')
else:
  # API call failed
  print(response.text)

INFO-__main__:6-Successful access token generation


In [6]:
#Combinations of destination and origin airport codes
origins = config.params['originLocationCode']
destinations = config.params['destinationLocationCode']
departure_dates=[departure_date_1_month,departure_date_2_month]
# return_dates=[return_date_1_month,return_date_2_month]

Creating combinations of Origin and Destination to be passed as parameters to the api call

trial

In [7]:
headers = {
  'client_id': config.client_id, 
  'client_secret': config.client_secret,
  'Authorization': f'Bearer {token}'
  }

url = config.url

response = []
counter = 0

for origin, dest, depart in itertools.product(origins, destinations, departure_dates):

    params={
        'originLocationCode': origin,
        'destinationLocationCode':dest, 
        'departureDate': depart, 
        'returnDate' : None,
        'adults':1,
        'children':None,
        'infants':None,
        'travelClass':None,
        'currencyCode':'EUR',
        'maxPrice' : None
        }


    #to be worked on later for by calling rest_api module
    # api = Api()
    # resp = api.make_flight_api_request(url,headers,params)
    resp = requests.get(url, headers=headers, params=params)
    counter = counter+1
    resp = resp.json()
    response.append(resp)
    logger.info(f'API call {counter} succeeded for flight offers')
    # else:
    #     # API call failed
    #     logger.info('API call failed')


INFO-__main__:35-API call 1 succeeded for flight offers
INFO-__main__:35-API call 2 succeeded for flight offers
INFO-__main__:35-API call 3 succeeded for flight offers
INFO-__main__:35-API call 4 succeeded for flight offers
INFO-__main__:35-API call 5 succeeded for flight offers
INFO-__main__:35-API call 6 succeeded for flight offers
INFO-__main__:35-API call 7 succeeded for flight offers
INFO-__main__:35-API call 8 succeeded for flight offers
INFO-__main__:35-API call 9 succeeded for flight offers
INFO-__main__:35-API call 10 succeeded for flight offers
INFO-__main__:35-API call 11 succeeded for flight offers
INFO-__main__:35-API call 12 succeeded for flight offers
INFO-__main__:35-API call 13 succeeded for flight offers
INFO-__main__:35-API call 14 succeeded for flight offers
INFO-__main__:35-API call 15 succeeded for flight offers
INFO-__main__:35-API call 16 succeeded for flight offers
INFO-__main__:35-API call 17 succeeded for flight offers
INFO-__main__:35-API call 18 succeeded f

### Saving the Global variable to a file

### Creating the payload for real time price script

In [16]:

# payload = {
#   "data": {
#     "type": "flight-offers-pricing",
#     "flightOffers": [], 
#     "travelers": []
#   }
# }

# for r in response:

#   # Extract flight offer data
#   offers = r['data']
  
#   # Add offers to payload
#   payload['data']['flightOffers'].extend(offers) 

# # Convert payload to JSON string for request 
# json_payload = json.dumps(payload)

In [17]:
# flight_search_response =json_payload

In [18]:
# Save the variable to a file
# with open('global_variable.pkl', 'wb') as file:
#     pickle.dump(flight_search_response, file)

Convert that sample nested JSON data into separate DataFrames

Repeat the same above procedure for itineraries and traveler_pricing

### Offers

In [64]:
offer_df = pd.DataFrame()

for i in range(24):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], record_path=['itineraries'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    offer_df = pd.concat([offer_df, data_to_append], ignore_index=True)

In [65]:
offer_df.head()

,duration,segments,id,source
0,PT10H35M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",1,GDS
1,PT14H40M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",2,GDS
2,PT16H25M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",3,GDS
3,PT23H30M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",4,GDS
4,PT25H15M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",5,GDS


Checking what the offer df contains

To get the first dept airport and last arrival airport looping through each row in itineraries df

In [66]:
offer_df['first_dept_airport'] = offer_df['segments'].apply(lambda x: x[0]['departure']['iataCode']) 
offer_df['last_arr_airport'] = offer_df['segments'].apply(lambda x: x[-1]['arrival']['iataCode'])
offer_df['first_dept_time'] = offer_df['segments'].apply(lambda x: x[0]['departure']['at']) 
offer_df['last_arr_time'] = offer_df['segments'].apply(lambda x: x[-1]['arrival']['at'])

Creating a composite key

Think About a unique key that you will use to join pricing table and segments table together to get the price of the flight

In [67]:
offer_df = offer_df.rename(columns={'id': 'offer_id'})

Trying another unique id

In [68]:
def generate_unique_id(row):
    unique_string = f"{row['offer_id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

offer_df['unique_id'] = offer_df.apply(generate_unique_id, axis=1)

In [69]:
offer_df=offer_df.drop(columns=['segments'],axis=1)

In [70]:
# Define the desired column order
desired_order = ['unique_id', 'offer_id', 'source','first_dept_airport','last_arr_airport','first_dept_time','last_arr_time','duration']

# Create a new DataFrame with columns in the desired order
offer_df = offer_df[desired_order]

In [72]:
offer_df.head()

,unique_id,offer_id,source,first_dept_airport,last_arr_airport,first_dept_time,last_arr_time,duration
0,e121ce1a8cadf04cefb01199ffb4c48b,1,GDS,BER,DEL,2023-12-04T11:50:00,2023-12-05T01:55:00,PT10H35M
1,31a29598b1799c34014de0155cd74d0c,2,GDS,BER,DEL,2023-12-04T06:00:00,2023-12-05T01:10:00,PT14H40M
2,f7fa70a52ca2d57ede7f7958a18e7261,3,GDS,BER,DEL,2023-12-04T06:00:00,2023-12-05T01:55:00,PT16H25M
3,5b280eeaeacff3508091c1e66f151c1a,4,GDS,BER,DEL,2023-12-04T21:10:00,2023-12-06T01:10:00,PT23H30M
4,ab2769fb5d649bebed5f4e3181ea70d5,5,GDS,BER,DEL,2023-12-04T21:10:00,2023-12-06T01:55:00,PT25H15M


### Itineraries segments

In [108]:
itineraries_df = pd.DataFrame()

for i in range(24):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], record_path=['itineraries'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    itineraries_df = pd.concat([itineraries_df, data_to_append], ignore_index=True)

In [110]:
itineraries_df['first_dept_airport'] = itineraries_df['segments'].apply(lambda x: x[0]['departure']['iataCode']) 
itineraries_df['last_arr_airport'] = itineraries_df['segments'].apply(lambda x: x[-1]['arrival']['iataCode'])
itineraries_df['first_dept_time'] = itineraries_df['segments'].apply(lambda x: x[0]['departure']['at']) 
itineraries_df['last_arr_time'] = itineraries_df['segments'].apply(lambda x: x[-1]['arrival']['at'])

In [111]:
itineraries_df.head()

,duration,segments,id,source,first_dept_airport,last_arr_airport,first_dept_time,last_arr_time
0,PT10H35M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",1,GDS,BER,DEL,2023-12-04T11:50:00,2023-12-05T01:55:00
1,PT14H40M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",2,GDS,BER,DEL,2023-12-04T06:00:00,2023-12-05T01:10:00
2,PT16H25M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",3,GDS,BER,DEL,2023-12-04T06:00:00,2023-12-05T01:55:00
3,PT23H30M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",4,GDS,BER,DEL,2023-12-04T21:10:00,2023-12-06T01:10:00
4,PT25H15M,"[{'departure': {'iataCode': 'BER', 'at': '2023...",5,GDS,BER,DEL,2023-12-04T21:10:00,2023-12-06T01:55:00


Trying to genearte unique id before exploding

In [112]:
def generate_unique_id(row):
    unique_string = f"{row['id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

itineraries_df['unique_id_fk'] = itineraries_df.apply(generate_unique_id, axis=1)

Treating the itineraries df

Just taking the exploded df and carefully scanning each row in the top 5 rows to see what kind of values does the exploded version of offer df contains
exploded df
df2
df

In [113]:
# Explode the 'segment' column to create separate rows for each dictionary
exploded_df = itineraries_df.explode('segments')

# Split the dictionary into columns  
df2 = exploded_df['segments'].apply(pd.Series)

# Merge the new columns back to the original DataFrame
df = exploded_df.merge(df2, right_index=True, left_index=True, suffixes=('_original', '_segments'))

# Drop the original dictionary column
df = df.drop(columns=['segments']) 
df = df.rename(columns={'duration_original': 'duration_total','id_original':'offer_id'})

In [115]:
# Extract departure keys using apply
df['dept_airport'] = df['departure'].apply(lambda x: x['iataCode'])
df['dept_at'] = df['departure'].apply(lambda x: x['at'])
df['arrival_airport'] = df['arrival'].apply(lambda x: x['iataCode'])
df['arrival_at'] = df['arrival'].apply(lambda x: x['at'])
df['aircraft'] = df['aircraft'].apply(lambda x: x['code'])
df.head()
# Drop stops column if it exists
column_to_drop = 'stops'
if column_to_drop in df.columns:
    df.drop(column_to_drop,axis=1,inplace=True)
itineraries_df = df.drop(columns=['departure','arrival','operating','aircraft','blacklistedInEU','first_dept_airport','last_arr_airport','first_dept_time','last_arr_time'])

itineraries_df = itineraries_df.drop_duplicates()
# Reset index
itineraries_df = itineraries_df.reset_index(drop=True)
itineraries_df.head()

,duration_total,offer_id,source,unique_id_fk,carrierCode,number,duration_segments,id_segments,numberOfStops,dept_airport,dept_at,arrival_airport,arrival_at
0,PT10H35M,1,GDS,e121ce1a8cadf04cefb01199ffb4c48b,KL,1824,PT1H25M,106,0,BER,2023-12-04T11:50:00,AMS,2023-12-04T13:15:00
1,PT10H35M,1,GDS,e121ce1a8cadf04cefb01199ffb4c48b,KL,109,PT8H15M,107,0,AMS,2023-12-04T14:10:00,DEL,2023-12-05T01:55:00
2,PT14H40M,2,GDS,31a29598b1799c34014de0155cd74d0c,KL,1818,PT1H25M,124,0,BER,2023-12-04T06:00:00,AMS,2023-12-04T07:25:00
3,PT14H40M,2,GDS,31a29598b1799c34014de0155cd74d0c,KL,871,PT8H20M,125,0,AMS,2023-12-04T12:20:00,DEL,2023-12-05T01:10:00
4,PT16H25M,3,GDS,f7fa70a52ca2d57ede7f7958a18e7261,KL,1818,PT1H25M,92,0,BER,2023-12-04T06:00:00,AMS,2023-12-04T07:25:00


In [117]:
itineraries_df['segment_key'] = itineraries_df['unique_id_fk'].astype(str) + '_' + itineraries_df['id_segments'].astype(str)


Create unique id for segments df

Renaming, reordering and changing datatypes

In [118]:
# Rename the columns according to the SQL table schema
itineraries_df = itineraries_df.rename(columns={
    'source': 'source',
    'duration_segments': 'duration_segments',
    'duration_total': 'duration_total',
    'carrierCode': 'carrier_code',
    'number': 'number',
    'duration_segments': 'duration_segments',
    'numberOfStops': 'number_of_stops',
    'dept_airport': 'dept_airport',
    'dept_at': 'dept_at',
    'arrival_airport': 'arrival_airport',
    'arrival_at': 'arrival_at'
})

# Ensure data types are set correctly
itineraries_df['offer_id'] = itineraries_df['offer_id'].astype(int)
itineraries_df['number_of_stops'] = itineraries_df['number_of_stops'].astype(int)
itineraries_df['dept_at'] = pd.to_datetime(itineraries_df['dept_at'])
itineraries_df['arrival_at'] = pd.to_datetime(itineraries_df['arrival_at'])

now = datetime.now()
itineraries_df['incremental_day']=now
itineraries_df['incremental_day'] = pd.to_datetime(itineraries_df['incremental_day'])


In [119]:
logger.info(f'Current date: {now}')

INFO-__main__:1-Current date: 2023-11-04 22:29:52.018577


In [120]:
itineraries_df.head()

,duration_total,offer_id,source,unique_id_fk,carrier_code,number,duration_segments,id_segments,number_of_stops,dept_airport,dept_at,arrival_airport,arrival_at,segment_key,incremental_day
0,PT10H35M,1,GDS,e121ce1a8cadf04cefb01199ffb4c48b,KL,1824,PT1H25M,106,0,BER,2023-12-04 11:50:00,AMS,2023-12-04 13:15:00,e121ce1a8cadf04cefb01199ffb4c48b_106,2023-11-04 22:29:52.018577
1,PT10H35M,1,GDS,e121ce1a8cadf04cefb01199ffb4c48b,KL,109,PT8H15M,107,0,AMS,2023-12-04 14:10:00,DEL,2023-12-05 01:55:00,e121ce1a8cadf04cefb01199ffb4c48b_107,2023-11-04 22:29:52.018577
2,PT14H40M,2,GDS,31a29598b1799c34014de0155cd74d0c,KL,1818,PT1H25M,124,0,BER,2023-12-04 06:00:00,AMS,2023-12-04 07:25:00,31a29598b1799c34014de0155cd74d0c_124,2023-11-04 22:29:52.018577
3,PT14H40M,2,GDS,31a29598b1799c34014de0155cd74d0c,KL,871,PT8H20M,125,0,AMS,2023-12-04 12:20:00,DEL,2023-12-05 01:10:00,31a29598b1799c34014de0155cd74d0c_125,2023-11-04 22:29:52.018577
4,PT16H25M,3,GDS,f7fa70a52ca2d57ede7f7958a18e7261,KL,1818,PT1H25M,92,0,BER,2023-12-04 06:00:00,AMS,2023-12-04 07:25:00,f7fa70a52ca2d57ede7f7958a18e7261_92,2023-11-04 22:29:52.018577


Segment key unique check

Finding out which columns in a dataframe are lists 

In [18]:
# Assuming deduplicated_df is your DataFrame
# list_columns = []

# for column in final_df.columns:
#     if final_df[column].apply(lambda x: isinstance(x, list)).any():
#         list_columns.append(column)

# print("Columns with lists:", list_columns)

Treating traveler_pricing_df

In [48]:
# Explode the 'segment' column to create separate rows for each dictionary
exploded_df = traveler_pricing_df.explode('fareDetailsBySegment')

# Apply json_normalize to the 'segment' column to split dictionaries into separate columns
normalized_df = json_normalize(exploded_df['fareDetailsBySegment']).add_suffix('_fare')

# Join the exploded and normalized DataFrames
final_df = exploded_df.drop('fareDetailsBySegment', axis=1).join(normalized_df)

# Drop duplicates based on all columns to deduplicate the DataFrame
deduplicated_df = final_df.drop_duplicates()

# # If you want to reset the index of the deduplicated DataFrame
deduplicated_df.reset_index(drop=True, inplace=True)
# deduplicated_df=deduplicated_df.drop(columns=['id_segments'])

# # Remove the added '_segment' suffix from column names
deduplicated_df.columns = deduplicated_df.columns.str.replace('_fare$', '', regex=True)

# #replace . in column anmes by underscore
deduplicated_df.columns = deduplicated_df.columns.str.replace('.', '_', regex=False)

# #Assigning to itineraries df
traveler_pricing_df = deduplicated_df

# #drop unwanted columns 
traveler_pricing_df=traveler_pricing_df.drop(columns=['fareBasis','class','brandedFare'])



---------------------------------------------------

Renaming, reordering and changing datatypes

In [49]:
# Rename the columns according to the SQL table schema
traveler_pricing_df = traveler_pricing_df.rename(columns={
    'travelerId': 'traveler_id',
    'fareOption': 'fare_option',
    'travelerType': 'traveler_type',
    'price_currency': 'price_currency',
    'price_total': 'price_total',
    'price_base': 'price_base',
    'id': 'id',
    'source': 'source',
    'segmentId': 'segment_id',
    'cabin': 'cabin',
    'includedCheckedBags_quantity': 'included_checkedbags_quantity',
    'includedCheckedBags_weight': 'included_checkedbags_weight',
    'includedCheckedBags_weightUnit': 'included_checkedbags_unit',
    'incremental_day': 'incremental_day'
})

# Change the data types of relevant columns
traveler_pricing_df['price_total'] = traveler_pricing_df['price_total'].astype(float)
traveler_pricing_df['price_base'] = traveler_pricing_df['price_base'].astype(float)
traveler_pricing_df['id'] = traveler_pricing_df['id'].astype(int)
traveler_pricing_df['segment_id'] = traveler_pricing_df['segment_id'].astype(int)
traveler_pricing_df['included_checkedbags_quantity'] = traveler_pricing_df['included_checkedbags_quantity'].astype(float)


# Ensure data types are set correctly
traveler_pricing_df['incremental_day'] =now
traveler_pricing_df['incremental_day'] = pd.to_datetime(traveler_pricing_df['incremental_day'])


In [51]:
credentials=config.pg_credentials
db = DBManager(credentials=credentials)
db.create_table_from_df(df=itineraries_df,table_name='itineraries')
db.create_table_from_df(df=traveler_pricing_df,table_name='traveler_pricing')

INFO-db_manager:27-CONNECTED
INFO-db_manager:73-Dumped to DB
INFO-db_manager:73-Dumped to DB
